In [ ]:
import numpy as np
import os
import pandas as pd
import sys
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, roc_auc_score, recall_score, precision_score,make_scorer
import seaborn as sns
sns.set(color_codes=True)
from scipy import stats
from scipy.stats import norm, skew #for some statistics
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
np.random.seed(25)
import os

In [ ]:
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['font.size'] = 15

In [ ]:
train = pd.read_csv("../input/train.csv",header=None)
test = pd.read_csv("../input/test.csv",header=None)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
header = pd.read_csv('../input/header.csv')
header.head()

In [ ]:
columns = train.columns
new_columns = []
for i in range(len(columns)):
    if i == 0:
        new_columns.append('key')
    elif i == (len(columns) - 1):
        new_columns.append('label')
    else:
        new_columns.append("V"+str(i))
    
train.columns = new_columns

In [ ]:
columns = test.columns
new_columns = []
for i in range(len(columns)):
    if i == 0:
        new_columns.append('key')
    else:
        new_columns.append("V"+str(i))
    
test.columns = new_columns

In [ ]:
train.isnull().sum(axis=0)

In [ ]:
train.duplicated().sum()

In [ ]:
sns.countplot(train['label'])

In [ ]:
# call regplot on each axes
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False)
sns.distplot(train['V1'],ax=ax1)
sns.distplot(np.log(train['V1']),ax=ax2)

In [ ]:
# call regplot on each axes
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False)
sns.distplot(train['V2'],ax=ax1)
sns.distplot(np.log(train['V2'] + 1),ax=ax2)

In [ ]:
# call regplot on each axes
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False)
sns.distplot(train['V3'],ax=ax1)
sns.distplot(np.log(train['V3'] + 1),ax=ax2)

In [ ]:
# call regplot on each axes
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False)
sns.distplot(train['V4'],ax=ax1)
sns.distplot(np.log(train['V4'] + 1),ax=ax2)

In [ ]:
# # call regplot on each axes
# fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False)
# sns.distplot(train['V5'],ax=ax1)
# sns.distplot(np.log(train['V5'] ),ax=ax2)

In [ ]:
# call regplot on each axes
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False)
sns.distplot(train['V6'],ax=ax1)
sns.distplot(np.log(train['V6'] + 1),ax=ax2)

In [ ]:
# call regplot on each axes
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False)
sns.distplot(train['V7'],ax=ax1)
sns.distplot(np.log(train['V7'] + 1),ax=ax2)

In [ ]:
# call regplot on each axes
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False)
sns.distplot(train['V8'],ax=ax1)
sns.distplot(np.log(train['V8'] + 1),ax=ax2)

In [ ]:
# call regplot on each axes
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False)
sns.distplot(train['V9'],ax=ax1)
sns.distplot(np.log(train['V9'] + 1),ax=ax2)

In [ ]:
# call regplot on each axes
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=False)
sns.distplot(train['V10'],ax=ax1)
sns.distplot(np.log(train['V10'] + 1),ax=ax2)

In [ ]:
col=train.columns
for i in col:
    if train[i].sum() == 0:
        print(i)

No Column with all zero values.

In [ ]:
#correlation matrix
corrmat = train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, square=True);

In [ ]:
# from patsy import dmatrices
# import statsmodels.api as sm
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# df = train[feature_names].copy()
# feature_names = [x for x in df.columns if x not in ['label']]
# #gather features
# features = "+".join(df[feature_names].columns)

# # get y and X dataframes based on this regression:
# y, X = dmatrices('label ~' + features, df, return_type='dataframe')

In [ ]:
# # For each X, calculate VIF and save in dataframe
# vif = pd.DataFrame()
# vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
# vif["features"] = X.columns

# vif.round(1)

In [ ]:
feature_names = [x for x in train.columns if x not in ['key', 'label','V7','V8','V9']]
target = train['label']

In [ ]:
# cat_columns = np.arange(7,51)
# cat_columns

In [ ]:
# model = ExtraTreesClassifier(n_estimators=500)
# ## model training and prediction
# model.fit(train[feature_names],target)
# pred1 = model.predict_proba(test[feature_names])[:,1]

In [ ]:
model = lgb.LGBMClassifier(device_type='gpu',boosting='dart',num_leaves=150,
                      learning_rate=0.05, n_estimators=10000, max_depth=15,
                      metric='auc',is_training_metric=True,
                      max_bin = 700, bagging_fraction = 0.8,verbose=-1,
                      bagging_freq = 5, feature_fraction = 0.80) ## model training and prediction
model.fit(train[feature_names],target)
pred2 = model.predict_proba(test[feature_names])[:,1]

In [ ]:
#model.feature_importances_

In [ ]:
pred = pred2#pred1 * 0.4 + pred2 * 0.6

In [ ]:
## make submission
sub = pd.DataFrame()
sub['key'] = test['key']
sub['score'] = pred
sub.to_csv('result.csv', index=False)

In [ ]:
sub.head()